# Travelling Salesperson Problems

Lucerne University of Applied Sciences and Arts - School of Information Technology

Code adapted from https://developers.google.com/optimization/routing/tsp/tsp

@author: Tobias Mérinat
@author: Marc Pouly

Imports

In [102]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
from tsp_examples import small, large, gps, randomly, drilling, manhattan

# A very simple Example

In [103]:
depot_index  = 0
num_vehicles = 1

city_names = ["New York", "Los Angeles", "Chicago", "Salt Lake City"]
                    
nodes = [
    [0, 2451, 713, 1018],   # New York
    [2451, 0, 1745, 1524],  # Los Angeles
    [713, 1745, 0, 355],    # Chicago
    [1018, 1524, 355, 0]]   # Salt Lake City)

def distance(a, b):
    return nodes[a][b]

routing = pywrapcp.RoutingModel(len(city_names), num_vehicles, depot_index)

routing.SetArcCostEvaluatorOfAllVehicles(distance)

assignment = routing.Solve()

if assignment:
    # Print objective, i.e. minimized total distance
    print('Total distance: {} miles'.format(assignment.ObjectiveValue()))
    
    # For each vehicle ...
    for vehicle_num in range(routing.vehicles()):
        result = []
        # Determine start index of the current vehicle
        index = routing.Start(vehicle_num)  
        while not routing.IsEnd(index):
            # Convert index to node and add to result
            result.append(city_names[routing.IndexToNode(index)])
            # Determine index of the next location
            index = assignment.Value(routing.NextVar(index))
        print(' - Vehicle {} travels {}'.format(vehicle_num + 1, result))
else:
    print('No solution found.')

Total distance: 5000 miles
 - Vehicle 1 travels ['New York', 'Chicago', 'Los Angeles', 'Salt Lake City']


# Larger and more complex Examples

Choose example

In [104]:
example = small # small refers to the same example as above

Assign values

In [105]:
city_names   = example.names
distance     = example.distance
num_cities   = example.num_cities
num_vehicles = example.num_vehicles
depot_index  = example.depot_index  

# Nodes are indexed from 0 to tsp_size - 1. The depot is the starting node of the route.

Create routing model

In [106]:
routing = pywrapcp.RoutingModel(num_cities, num_vehicles, depot_index)

All vehicles use the same cost model

In [107]:
routing.SetArcCostEvaluatorOfAllVehicles(distance)

#### Some additional constraints
Do not forget to switch to the large example and increase number of vehicles when necessary.

A vehicle must not visit more than 7 cities

In [108]:
#routing.AddConstantDimension(1, 7, True, "count")

A vehicle must visit at least 3 cities

In [109]:
#routing.AddConstantDimension(1, num_cities, True, "count")
#count = routing.GetDimensionOrDie("count")
#for i in range(num_vehicles):
#    count.CumulVar(routing.End(i)).SetMin(3)

City 2 and 4 (counting starts at 0) must NOT be visited by the same vehicle

In [110]:
#routing.solver().Add(routing.VehicleVar(2) != routing.VehicleVar(4))

City 2 must be visited before city 4 (not necessarily by the same vehicle)

In [111]:
# routing.AddConstantDimension(1, num_cities, True, "time")
# time = routing.GetDimensionOrDie("time")
# routing.solver().Add(time.CumulVar(2) < time.CumulVar(4))

City 2 must be visited right after city 4 (not necessarily by the same vehicle)

In [112]:
# routing.AddConstantDimension(1, num_cities, True, "time")
# time = routing.GetDimensionOrDie("time")
# routing.solver().Add(time.CumulVar(4) + 1 == time.CumulVar(2))

Configure solver

In [113]:
parameters = pywrapcp.RoutingModel.DefaultSearchParameters()
parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
parameters.time_limit_ms = 10000

Start solver

In [114]:
assignment = routing.SolveWithParameters(parameters)

Print solution

In [115]:
if assignment:
    print('Total distance: {} miles\n'.format(assignment.ObjectiveValue()))
    for vehicle_num in range(routing.vehicles()):
        print('Vehicle {}:'.format(vehicle_num+1))
        route = ""
        index = routing.Start(vehicle_num)  # index of the variable for the starting node
        while not routing.IsEnd(index):
            # Convert variable indices to node indices for displaying
            route += '  - {} -> '.format((city_names[routing.IndexToNode(index)]))
            index = assignment.Value(routing.NextVar(index))
            route += '{}\n'.format(city_names[routing.IndexToNode(index)])
        print(route)
else:
    print('No solution found.')

Total distance: 5000 miles

Vehicle 1:
  - New York -> Chicago
  - Chicago -> Los Angeles
  - Los Angeles -> Salt Lake City
  - Salt Lake City -> New York

